In [2]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np

#### Colocando os dados em um DataFrame

In [3]:
tw = pd.read_excel('tweets_lindt.xlsx')

In [4]:
tw.head()

,Treinamento,Relevante,Positivo/Negativo
0,lindt chocolate is so overrated,1,0
1,@elgliko hear that pitter patter in the back g...,0,NaN
2,rt @statutory_boy: the guy was addicted to dru...,0,NaN
3,"@andreyasasylum @yourmomsuckstho hey, can you ...",1,1
4,great job by @noticgroup gms. come and see the...,1,1


#### Mudança de índice

A mudança de iíndice foi realizada para que a primeira coluna do DataFrame fossem os tweets

In [5]:
tw_idx = tw.set_index('Treinamento')
tw_idx

,Relevante,Positivo/Negativo
Treinamento,,
lindt chocolate is so overrated,1,0
"@elgliko hear that pitter patter in the back ground every so often?\nthat’s me.\nin your costco.\nwhere i am living off a rounded diet of slim jim’s &amp; lindt chocolate, reading romance novels by the 6-pack, and having my pick of the giant pile of dog beds for some shuteye.\nyou’ll only find me",0,NaN
"rt @statutory_boy: the guy was addicted to drugs, his life is changed after he met this woman, god bless this woman ❤👊🙏 https://t.co/dcnehu…",0,NaN
"@andreyasasylum @yourmomsuckstho hey, can you please pass the last of that lindt dark chocolate?",1,1
great job by @noticgroup gms. come and see them and make sure you ask for a lindt ball. \n#signaturesolutions \n#wearecbr https://t.co/ynsmdfcxaj,1,1
rt @blacktwittasa_: lmao guys i'm screaming 🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣 https://t.co/sx7w2cfmvx,0,NaN
fuck i want candy or chocolate\n\ngoing ✈️ to spend all my money at the lindt store,1,1
@lindt_zwane thank you,0/1,NaN
@lindt_zwane misinterpret of of love sex money 💵 reliability patriarchy just ruined the whole meaning of true love,0,NaN


#### Separando Relevante e Irrelevante

In [7]:
rel = tw_idx[tw_idx.Relevante == 1]
rel

,Relevante,Positivo/Negativo
Treinamento,,
lindt chocolate is so overrated,1,0
"@andreyasasylum @yourmomsuckstho hey, can you please pass the last of that lindt dark chocolate?",1,1
great job by @noticgroup gms. come and see them and make sure you ask for a lindt ball. \n#signaturesolutions \n#wearecbr https://t.co/ynsmdfcxaj,1,1
fuck i want candy or chocolate\n\ngoing ✈️ to spend all my money at the lindt store,1,1
reebok emailed me to thank me for my collab posts with their shoes nothing else matters today\n\nexcept that lindt sent me ten truffle bags who wants to turn up w me,1,1
"😥 news - coffee, irish cream &amp; hazelnut gianduja lindt lindor's discontinued - get what's left while you can! https://t.co/ncphz2ypb7 https://t.co/hfslcab317",1,1
"@earwolf @mattgourley @tvsandydaly a new august lindt! whooohoooo! finally, and end to my sufferings!",1,1
rt @gemsatwork: is this real life? 😍we are over the moon to be working with lindt again. what an amazing way to distract us from post-summe…,1,1
so kira asked me to buy her chocolates - lindt has this pick and mix shop where you can take as much as you like as long as you can fit it in the box. level: asian 🤣 https://t.co/2lumsa7bdp,1,1


In [8]:
irrel = tw_idx[tw_idx.Relevante == 0]
irrel

,Relevante,Positivo/Negativo
Treinamento,,
"@elgliko hear that pitter patter in the back ground every so often?\nthat’s me.\nin your costco.\nwhere i am living off a rounded diet of slim jim’s &amp; lindt chocolate, reading romance novels by the 6-pack, and having my pick of the giant pile of dog beds for some shuteye.\nyou’ll only find me",0,NaN
"rt @statutory_boy: the guy was addicted to drugs, his life is changed after he met this woman, god bless this woman ❤👊🙏 https://t.co/dcnehu…",0,NaN
rt @blacktwittasa_: lmao guys i'm screaming 🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣 https://t.co/sx7w2cfmvx,0,NaN
@lindt_zwane misinterpret of of love sex money 💵 reliability patriarchy just ruined the whole meaning of true love,0,NaN
how much out of 10 would you rate the 70% #darkchocolate version of #lindtexcellence blocks? https://t.co/aeppg6mlu4,0,NaN
"rt @curiouspeterg: how are unicorns fake but giraffes are real? like whats more believable, a horse with a horn or a leopard-moose-camel wi…",0,NaN
rt @crumbschester: 🍰 we are heading back to the @_makersmarket in chester this month. we’ve been working hard on new tasty treats lately &amp;…,0,NaN
co. i find them so sexy and intriguing. https://t.co/n5adazxwoa,0,NaN
rt @_bonga: guys. please put on your seatbelts when driving. i’ve just been informed that an old friend passed away yesterday morning after…,0,NaN


#### Criando as listas

In [10]:
lista_rel = []
lista_rel.append(rel)
lista_rel

[                                                   Relevante Positivo/Negativo
 Treinamento                                                                   
 lindt chocolate is so overrated                            1                 0
 @andreyasasylum @yourmomsuckstho hey, can you p...         1                 1
 great job by @noticgroup gms. come and see them...         1                 1
 fuck i want candy or chocolate\n\ngoing ✈️ to s...         1                 1
 reebok emailed me to thank me for my collab pos...         1                 1
 😥 news - coffee, irish cream &amp; hazelnut gia...         1                 1
 @earwolf @mattgourley @tvsandydaly a new august...         1                 1
 rt @gemsatwork: is this real life? 😍we are over...         1                 1
 so kira asked me to buy her chocolates - lindt ...         1                 1
 rt @piwaaiko_xo: @lindt_zwane lmao nope😂😂\nthan...         1                 0
 #previous #post  the sensational taste 